In [1]:
# import required packages
import os
from time import sleep
from random import randint
import pandas as pd
import newspaper
from bs4 import BeautifulSoup
import requests as rq
import json
import nltk
import ssl
import calendar
import time
from newspaper import Article
from newspaper import Config

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\steni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Manual creation get_urls_of_captures list due to the archive api being down 
def manual_get_urls_of_captures(): 
    #01.01.2022 - 31.01.2022
    final_url_list = [
        "https://web.archive.org/web/20220101101506/http://www.bbc.com/", 
        "https://web.archive.org/web/20220102101510/http://www.bbc.com/",
        "https://web.archive.org/web/20220103101535/https://www.bbc.com/",
        "https://web.archive.org/web/20220104104015/https://www.bbc.com/",
        "https://web.archive.org/web/20220105111551/http://www.bbc.com/",
        "https://web.archive.org/web/20220106111553/http://www.bbc.com/",
        "https://web.archive.org/web/20220107111707/https://www.bbc.com/",
        "https://web.archive.org/web/20220108113014/http://www.bbc.com/",
        "https://web.archive.org/web/20220109121526/https://www.bbc.com/",
        "https://web.archive.org/web/20220110121539/http://www.bbc.com/",
        "https://web.archive.org/web/20220111122739/https://www.bbc.com/",
        "https://web.archive.org/web/20220112124750/https://www.bbc.com/",
        "https://web.archive.org/web/20220113131043/https://www.bbc.com/",
        "https://web.archive.org/web/20220114131546/http://www.bbc.com/",
        "https://web.archive.org/web/20220115131717/http://www.bbc.com/",
        "https://web.archive.org/web/20220116141608/http://www.bbc.com/",
        "https://web.archive.org/web/20220117141525/https://www.bbc.com/",
        "https://web.archive.org/web/20220118151542/http://www.bbc.com/",
        "https://web.archive.org/web/20220119161414/http://www.bbc.com/",
        "https://web.archive.org/web/20220120163833/https://www.bbc.com/",
        "https://web.archive.org/web/20220121171555/http://www.bbc.com/",
        "https://web.archive.org/web/20220122171711/https://www.bbc.com/",
        "https://web.archive.org/web/20220123171737/http://www.bbc.com/",
        "https://web.archive.org/web/20220124173041/https://www.bbc.com/",
        "https://web.archive.org/web/20220125174040/https://www.bbc.com/",
        "https://web.archive.org/web/20220126174110/https://www.bbc.com/",
        "https://web.archive.org/web/20220127175131/https://www.bbc.com/",
        "https://web.archive.org/web/20220128175546/https://www.bbc.com/",
        "https://web.archive.org/web/20220129181124/https://www.bbc.com/",
        "https://web.archive.org/web/20220130182141/https://www.bbc.com/", 
        "https://web.archive.org/web/20220131183052/https://www.bbc.com/",
        ]
    return final_url_list


In [3]:
# collect all urls for captures by the wayback machine in a specific timeframe
def get_urls_of_captures():
    base_url ="http://web.archive.org/cdx/search/cdx"

    # change following values to modify the request
    url_of_interest = "bbc.com"
    from_date = "20220101"
    to_date = "20220201" # include first of next month to be sure to scrape all articles of last day
    output_format ="json"

    # set final url together
    url = f"{base_url}?url={url_of_interest}&collapse=digest&from={from_date}&to={to_date}&output={output_format}"

    # request urls of captures from wayback cdx api
    urls_of_captures = rq.get(url).text
    parse_urls_of_captures = json.loads(urls_of_captures) 

    # Extracts timestamp and original columns from urls and compiles a url list
    url_list = []
    for i in range(1,len(parse_urls_of_captures)): # start at 1 to skip headers
        orig_url = parse_urls_of_captures[i][2]
        timestamp = parse_urls_of_captures[i][1]
        waylink = timestamp+'/'+orig_url
        url_list.append(waylink)

    # Compile final url pattern
    final_urls_of_captures = []
    for url in url_list:
        final_url = 'https://web.archive.org/web/'+url
        final_urls_of_captures.append(final_url)

    return final_urls_of_captures



In [4]:
# helper to delete cache of the newspaper3k library
def delete_feed_category_cache_path():
    # the following location of the feed_category_cache might be different depending on operating system
    # current location is the filepath to the feed_category_cache on windows
    feed_category_cache_path = r"C:\Users\steni\AppData\Local\Temp\.newspaper_scraper\feed_category_cache"

    # delete all files inside the cache
    for filename in os.listdir(feed_category_cache_path):
        file_path = os.path.join(feed_category_cache_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [5]:
def clean_scraped_date(data):
    cleaned_data = []
    for item in data:
        item = item.string.lower()
        if item not in cleaned_data:
            cleaned_data.append(item)
    return cleaned_data

In [6]:
urls_of_captures = get_urls_of_captures()

In [7]:
len(urls_of_captures)

3780

In [17]:
counter_url = 0
collection_of_scraped_articles = []
collection_of_failed_scraped_articles = []
scraped_urls = []

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10


for url in urls_of_captures[0:1]:
    print("url", counter_url)
    counter_url += 1
    delete_feed_category_cache_path() # throws error if path doesn't exists

    counter = 0
    
    # building the news source
    bbc = newspaper.build(url, memoize_articles=False, language='en')

    for article in bbc.articles:
        try:
            print("article", counter)
            counter += 1
            article_url = article.url
            
            main_article_url = article_url.split("bbc.com")[1]
            if main_article_url in scraped_urls:
                continue

            sleep(randint(5, 8))

            page = Article(article_url, config=config)
            page.download()
            article_meta_data = page.meta_data

            # collect specific data of article
            soup = BeautifulSoup(page.html, 'html.parser')
            page.parse()
            bbc_dictionary = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))
            
            #find date
            article_date = [value for (key, value) in bbc_dictionary.items() if key == 'datePublished']

            # find ul list containing tags
            parent = soup.find("section").find("ul")
            article_keywords = list(parent.descendants)
            
            # find author
            article_authors = [value['name'] for (key, value) in bbc_dictionary.items() if key == 'author']
            
            # collect title and summary
            article_title = [value for (key, value) in bbc_dictionary.items() if key == 'headline']
            text = soup.find_all("p", {"class": "ssrcss-1q0x1qg-Paragraph eq5iqo00"})
            article_text = ""
            for element in text:
                article_text += "\n" + "".join(element.findAll(string = True)) 

            # clean up scraped data
            if not article_title or article_title == None:
                article_title = "N/A"
            if not article_text or article_text == None:
                article_text = "N/A"
            if not article_url or article_url == None:
                article_url = "N/A"
            if article_date:
                article_date = article_date
            if article_keywords and article_keywords != None:
                article_keywords = clean_scraped_date(article_keywords)
            if article_authors and article_authors != None:
                article_authors = article_authors
            
            # use nlp to get keywords of article
            page.nlp()
            article_nlp_keywords = page.keywords
            if not article_nlp_keywords:
                article_nlp_keywords = "N/A"
            
            collection_of_scraped_articles.append([article_title, 
                                                    article_date,
                                                    article_authors,
                                                    article_keywords,
                                                    article_nlp_keywords,
                                                    article_text,
                                                    article_url])
            scraped_urls.append(main_article_url)
            sleep(randint(5, 8))
        
        except Exception as error:
            collection_of_failed_scraped_articles.append([article_url, error])
            continue
    
# create data frames
scraped_articles_df = pd.DataFrame(collection_of_scraped_articles, columns=["Title", "Date", "Authors", "Tags", "NLP Keywords", "text", "Url"])
failed_articles_df = pd.DataFrame(collection_of_failed_scraped_articles, columns=["Url", "Error"])

# save data frames as csv with current timestamp
current_GMT = time.gmtime()
time_stamp = calendar.timegm(current_GMT)
scraped_articles_df.to_csv(f'{time_stamp}_articles.csv', sep=';')
failed_articles_df.to_csv(f'{time_stamp}_failed.csv', sep=';')

url 0
article 0
article 1
article 2
article 3
article 4
article 5
article 6
article 7
article 8
article 9
article 10
article 11
article 12
article 13
article 14
article 15
article 16
article 17
article 18
article 19
article 20
article 21
article 22
article 23
article 24
article 25
article 26
article 27
article 28
article 29
article 30
article 31
article 32
article 33
article 34
article 35
article 36
article 37
article 38
article 39
article 40
article 41
article 42
article 43
article 44
article 45
article 46
article 47
article 48
article 49
article 50
article 51
article 52
article 53
article 54
article 55
article 56
article 57
article 58
article 59
article 60
article 61
article 62
article 63
article 64
article 65
article 66
article 67
article 68
article 69
article 70
article 71
article 72
article 73
article 74
article 75
article 76
article 77
article 78
article 79
article 80
article 81
article 82
article 83
article 84
article 85
article 86
article 87
article 88
article 89


In [18]:
print(len(failed_articles_df))
print(len(scraped_articles_df))

56
34


In [23]:
a = scraped_articles_df.head(1)

,Title,Date,Authors,Tags,NLP Keywords,text,Url
0,[Covid-19: WHO chief optimistic disease will b...,[2021-12-31T19:28:26.000Z],[BBC News],"[[[World Health Organization (WHO)]], [World H...","[missed, beaten, disease, dose, optimistic, ra...",\nThis video can not be played\nThe World Heal...,https://web.archive.org/web/20220101000057/htt...
